# Week 5 Assignment
## 3 Questions to answer:
1. What is the northernmost airport in the United States?
2. What is the easternmost airport in the United States?
3. On February 12th, 2013, which New York area airport had the windiest weather?

## 1. What is the northernmost airport in the United States?

### First, I'll take a look at the data using .head to see the first 5 rows

In [5]:
import numpy as np
import pandas as pd

url = 'https://github.com/tidyverse/nycflights13/blob/main/data-raw/airports.csv?raw=true'
airports = pd.read_csv(url, index_col=0)

print(airports.head(5))

                              name        lat        lon   alt  tz dst  \
faa                                                                      
04G              Lansdowne Airport  41.130472 -80.619583  1044  -5   A   
06A  Moton Field Municipal Airport  32.460572 -85.680028   264  -6   A   
06C            Schaumburg Regional  41.989341 -88.101243   801  -6   A   
06N                Randall Airport  41.431912 -74.391561   523  -5   A   
09J          Jekyll Island Airport  31.074472 -81.427778    11  -5   A   

                tzone  
faa                    
04G  America/New_York  
06A   America/Chicago  
06C   America/Chicago  
06N  America/New_York  
09J  America/New_York  


### Immediately, I have a couple of questions about the data. Are all the airports in this file in the U.S.? How many airports are listed here? These are things I will need to know in order to get an accurate answer, and also one which I can validate with research.

In [8]:
airports.groupby('tzone').name.count()

tzone
America/Anchorage      239
America/Chicago        342
America/Denver         119
America/Los_Angeles    176
America/New_York       519
America/Phoenix         38
America/Vancouver        2
Asia/Chongqing           2
Pacific/Honolulu        18
Name: name, dtype: int64

### I can already see there are 4 airports outside of the U.S. (and many more outside of the contiguous states), but now I am interested to know if there are any airports without timezones.

In [9]:
airports.count()

name     1458
lat      1458
lon      1458
alt      1458
tz       1458
dst      1458
tzone    1455
dtype: int64

### Aha! I can see there are 3 airports without timezones, and I want to know which ones those are before I start making assumptions about the northernmost U.S. airport. At least now I know that there are 1454 airports listed with time zones in the U.S. in this data set. I can use that for comparison pursposes later during research.

In [19]:
airports[airports.isna().any(axis=1)]

,name,lat,lon,alt,tz,dst,tzone
faa,,,,,,,
EEN,Dillant Hopkins Airport,72.270833,42.898333,149,-5,A,NaN
LRO,Mount Pleasant Regional-Faison Field,32.538700,-79.469700,12,-5,A,NaN
YAK,Yakutat,59.301200,-139.393700,33,-9,A,NaN


### I looked up these specific airports and see that EEN is in NH, LRO is in SC, and YAK is in AK.

### Because I am including HI and AK in this analysis, all 3 of these airports are valid. I will only exclude the 2 Asia time zone airports. 

### Now I will use this information to find out what the northernmost airport is.

In [30]:
airports[airports['tzone'] != 'Asia/Chongqing'].lat.max()

72.270833

In [31]:
airports[airports['lat'] == 72.270833]

,name,lat,lon,alt,tz,dst,tzone
faa,,,,,,,
EEN,Dillant Hopkins Airport,72.270833,42.898333,149,-5,A,NaN


### New Hampshire is pretty 'north' but I would have expected the answer to be somewhere in Alaska. I'm going to check out what the data says about Alaska airports and their lat/lon coordinates.

In [32]:
airports[airports['tzone'] == 'America/Anchorage']

,name,lat,lon,alt,tz,dst,tzone
faa,,,,,,,
369,Atmautluak Airport,60.866667,-162.273056,18,-9,A,America/Anchorage
6K8,Tok Junction Airport,63.329444,-142.953611,1639,-9,A,America/Anchorage
ABL,Ambler Airport,67.106389,-157.857500,334,-9,A,America/Anchorage
ADK,Adak Airport,51.878000,-176.646000,18,-9,A,America/Anchorage
ADQ,Kodiak,57.749967,-152.493856,78,-9,A,America/Anchorage
...,...,...,...,...,...,...,...
WTK,Noatak Airport,67.566100,-162.975000,88,-9,A,America/Anchorage
WTL,Tuntutuliak Airport,60.335278,-162.666944,16,-9,A,America/Anchorage
WWP,North Whale Seaplane Base,56.116389,-133.121667,0,-9,A,America/Anchorage


### Hmmm.... either coordinates are wrong for Alaska, or they are wrong for New Hampshire. After doing some searching, I found that it looks like the EEN airport had its coordinates switched. The latitude is actually 42.89, not the longitude.

### I think it's safe to say that I should be evaluating all Alaska time zone airports, with the addition of YAK.

In [33]:
airports[airports['tzone'] == 'America/Anchorage'].lat.max()

71.285446

In [36]:
airports[airports['lat'] == 71.285446]

,name,lat,lon,alt,tz,dst,tzone
faa,,,,,,,
BRW,Wiley Post Will Rogers Mem,71.285446,-156.766003,44,-9,A,America/Anchorage


## Answer 1: The northernmost airport is BRW.

## 2. What is the easternmost airport in the U.S.?

### I will start by using the same logic as the last question and see what that gets me.

In [37]:
airports[airports['tzone'] != 'Asia/Chongqing'].lon.max()

174.11362

In [38]:
airports[airports['lon'] == 174.11362]

,name,lat,lon,alt,tz,dst,tzone
faa,,,,,,,
SYA,Eareckson As,52.712275,174.11362,98,-9,A,America/Anchorage


### This is obviously a mistake. The Alaska longitudes are negative, so this must be an error.

### I will try again, exclusing that time zone.

In [49]:
airports[(airports['tzone'] != 'America/Anchorage') & (airports['tzone'] != 'Asia/Chongqing')].lon.max()

42.898333

In [50]:
airports[airports['lon'] == 42.898333]

,name,lat,lon,alt,tz,dst,tzone
faa,,,,,,,
EEN,Dillant Hopkins Airport,72.270833,42.898333,149,-5,A,NaN


### Well, now I got the airport that I already established has the lat and lon mixed up.

### Maybe I should go ahead and fix these errors.

In [51]:
airports.loc[(airports.name == 'Dillant Hopkins Airport'), 'lat'] = 42.898333
airports.loc[(airports.name == 'Dillant Hopkins Airport'), 'lon'] = 72.270833
print(airports[airports.name == 'Dillant Hopkins Airport'])

                        name        lat        lon  alt  tz dst tzone
faa                                                                  
EEN  Dillant Hopkins Airport  42.898333  72.270833  149  -5   A   NaN


In [52]:
airports.loc[(airports.name == 'Eareckson As'), 'lon'] = -174.11362
print(airports[airports.name == 'Eareckson As'])

             name        lat        lon  alt  tz dst              tzone
faa                                                                    
SYA  Eareckson As  52.712275 -174.11362   98  -9   A  America/Anchorage


### Okay, let's try this again.

In [53]:
airports[airports['tzone'] != 'Asia/Chongqing'].lon.max()

72.270833

## Answer 2: The easternmost airport is Dillant Hopkins Airport in New Hampshire

### Now I'm supposed to figure out which New York area airport had the windiest weather on February 12th, 2013.
### Something tells me I'm going to need to join some tables, but I'll find out after looking at the first few rows of the weather .csv file.

In [55]:
url2 = 'https://github.com/tidyverse/nycflights13/blob/main/data-raw/weather.csv?raw=true'
weather = pd.read_csv(url2, index_col=0)

print(weather.head(5))

        year  month  day  hour   temp   dewp  humid  wind_dir  wind_speed  \
origin                                                                      
EWR     2013      1    1     1  39.02  26.06  59.37     270.0    10.35702   
EWR     2013      1    1     2  39.02  26.96  61.63     250.0     8.05546   
EWR     2013      1    1     3  39.02  28.04  64.43     240.0    11.50780   
EWR     2013      1    1     4  39.92  28.04  62.21     250.0    12.65858   
EWR     2013      1    1     5  39.02  28.04  64.43     260.0    12.65858   

        wind_gust  precip  pressure  visib             time_hour  
origin                                                            
EWR           NaN     0.0    1012.0   10.0  2013-01-01T06:00:00Z  
EWR           NaN     0.0    1012.3   10.0  2013-01-01T07:00:00Z  
EWR           NaN     0.0    1012.5   10.0  2013-01-01T08:00:00Z  
EWR           NaN     0.0    1012.2   10.0  2013-01-01T09:00:00Z  
EWR           NaN     0.0    1011.9   10.0  2013-01-01T10:

### None of the columns are location-based, so now I'm assuming that all of these are in the NY area.
### Perhaps I will find the windiest weather first, and then establish if that airport is in the NY area.

In [63]:
weather.sort_values('wind_speed')

,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
origin,,,,,,,,,,,,,,
EWR,2013,7,17,14,93.02,66.02,41.20,0.0,0.00000,NaN,0.0,1019.7,10.0,2013-07-17T18:00:00Z
EWR,2013,5,22,21,73.94,64.04,71.22,0.0,0.00000,NaN,0.0,1011.6,10.0,2013-05-23T01:00:00Z
LGA,2013,11,6,3,48.02,41.00,76.53,0.0,0.00000,NaN,0.0,1031.1,10.0,2013-11-06T08:00:00Z
LGA,2013,11,6,2,50.00,41.00,71.07,0.0,0.00000,NaN,0.0,1031.5,10.0,2013-11-06T07:00:00Z
LGA,2013,11,6,1,48.92,39.92,70.95,0.0,0.00000,NaN,0.0,1032.0,10.0,2013-11-06T06:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EWR,2013,2,12,3,39.02,26.96,61.63,260.0,1048.36058,NaN,0.0,1008.3,10.0,2013-02-12T08:00:00Z
EWR,2013,3,27,17,51.98,19.04,27.02,NaN,NaN,NaN,0.0,1012.1,10.0,2013-03-27T21:00:00Z
JFK,2013,5,22,10,62.06,59.00,93.79,NaN,NaN,NaN,0.0,NaN,2.5,2013-05-22T14:00:00Z


### It looks like the windiest weather was at EWR with a wind speed of 1048.36058, but I should get rid of the NaN values first

In [68]:
weather.dropna(subset='wind_speed').sort_values('wind_speed')

,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
origin,,,,,,,,,,,,,,
EWR,2013,7,17,14,93.02,66.02,41.20,0.0,0.00000,NaN,0.00,1019.7,10.0,2013-07-17T18:00:00Z
EWR,2013,5,22,21,73.94,64.04,71.22,0.0,0.00000,NaN,0.00,1011.6,10.0,2013-05-23T01:00:00Z
LGA,2013,11,6,3,48.02,41.00,76.53,0.0,0.00000,NaN,0.00,1031.1,10.0,2013-11-06T08:00:00Z
LGA,2013,11,6,2,50.00,41.00,71.07,0.0,0.00000,NaN,0.00,1031.5,10.0,2013-11-06T07:00:00Z
LGA,2013,11,6,1,48.92,39.92,70.95,0.0,0.00000,NaN,0.00,1032.0,10.0,2013-11-06T06:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EWR,2013,1,31,4,60.80,59.00,93.79,230.0,40.27730,58.68978,0.30,NaN,6.0,2013-01-31T09:00:00Z
LGA,2013,1,31,4,59.00,55.40,93.69,230.0,40.27730,55.23744,0.41,NaN,8.0,2013-01-31T09:00:00Z
JFK,2013,1,31,4,53.60,53.06,100.00,200.0,42.57886,52.93588,0.20,NaN,2.0,2013-01-31T09:00:00Z


### I just realized that there are different months and days in this dataframe (oops).
### However, the windiest weather overall just happened to be on the date in question, Feb 12th, 2013.

## Answer 3: The windiest weather on February 12, 2013 was at EWR airport.

## Final Answers:
1. BRW
2. EEN
3. EWR